In [4]:
md_reference_data = None
upstream = None
product = None

In [5]:
# declare a list tasks whose products you want to use as inputs
upstream = None


# R23 - MD alignment and RMSD extraction 

## Contents

1. Alignment
2. RMSD
3. RMSD against average structure
4. RMSD over subunit
5. RMSF

## Alignment and preparation

In [1]:
from mdtools.mapping import map_alignment_to_structure, align_structure_sequences
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align
from pyfamsa import Aligner, Sequence
import json
import pandas as pd
import prody as pdy
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import prody as pdy
import MDAnalysis as mda
# from pyfamsa import Aligner, Sequence
from yaml import load, Loader
from Bio import AlignIO
from Bio.SeqUtils import seq1
from Bio.SeqIO import SeqRecord
from Bio import SeqIO
from Bio.Seq import Seq
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA as sk_pca
from MDAnalysis.analysis import align
import pickle
import tqdm
plt.rcParams['svg.fonttype'] = 'none'
#matplotlib.rcParams['fontname'] = "Arial"

/home/kaustubh/miniconda3/envs/md-analysis/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
md_reference_data = load(open("../env_rep2.yaml"), Loader)['experiment_01']

In [3]:
def load_trajectory(code, data, path):
    
    """
    loads the following files
    - an MDAnalysis trajectory with the dried (water-removed) dcd
    - an MDAnalysis Universe with the starting PDB (jic)
    - an prody with the starting PDB (pretty useful when MDAnalysis fails)
    """
    return dict(
        code=code,
        trajectory_dry = mda.Universe(
            path + data['dry_pdb'], 
            path + data['dry_dcd'], 
            frames='all', in_memory=True
        ),
        reference_pdy=pdy.parsePDB(path + data['dry_pdb']),
        reference=mda.Universe(
            path + data['dry_pdb']
        ),
        time=data['time']
    )


Loading all the simulations.

In [4]:
path = '/home/kaustubh/MD_Simulations/simulations_data/'
exp01_md = []
for key, items in md_reference_data.items():
    print(f"-- code {key}", end='')
    exp01_md.append(load_trajectory(
        code=key, data=items, path=path
    ))
    print(f" loaded")
exp01_md = pd.DataFrame.from_records(exp01_md).set_index('code')

-- code 1bwv

@> 77672 atoms and 1 coordinate set(s) were parsed in 0.37s.


 loaded
-- code 3zxw

@> 69984 atoms and 1 coordinate set(s) were parsed in 0.34s.


 loaded
-- code 6ftl

...

 loaded
-- code ancip

@> 54768 atoms and 1 coordinate set(s) were parsed in 0.26s.


 loaded
-- code anci-wo-ssu

@> 58584 atoms and 1 coordinate set(s) were parsed in 0.28s.


 loaded


## RMSD calculations

We compute the RMSD of:
- All the alpha carbons.
- Large subunit alpha carbons
- Small subunit alpha carbons
- Upper and lower large subunit alpha carbons
- Upper and lower small subunit alpha carbons

In [5]:
exp01_rmsd = []

complex_all = 'name CA'
upper_ssu = 'name CA and (segid B or segid F or segid J or segid N)'
lower_ssu = 'name CA and (segid D or segid H or segid L or segid P)'
ssu = 'name CA and (segid B or segid F or segid J or segid N or segid D or segid H or segid L or segid P)'
upper_lsu = 'name CA and (segid A or segid E or segid I or segid M)'
lower_lsu = 'name CA and (segid C or segid G or segid K or segid O)'
lsu = 'name CA and (segid A or segid E or segid I or segid M or segid C or segid G or segid K or segid O)'


for key, item in exp01_md.iterrows():
    print(f"-- code {key}", end='')
    R = rms.RMSD(item['trajectory_dry'], item['reference'], select='name CA', groupselections=[upper_ssu, lower_ssu, ssu, upper_lsu, lower_lsu, lsu], superposition=False)
    results = R.run()
    tmp = pd.DataFrame(results.rmsd, columns=['frame', 'time', 'CA', 'upperSSU', 'lowerSSU', 'ssu', 'upperLSU', 'lowerLSU', 'lsu'])
    tmp['code'] = key
    exp01_rmsd.append(tmp)
    print(f" aligned")
    
exp01_rmsd = pd.concat(exp01_rmsd)
exp01_rmsd.index = pd.MultiIndex.from_arrays([exp01_rmsd['code'], exp01_rmsd['frame']])
exp01_rmsd['frame'] = exp01_rmsd['frame'].astype(int)
exp01_rmsd['time'] = exp01_rmsd['time'].astype(float)
exp01_rmsd['time_ns'] = exp01_rmsd['frame'] / 2.5

-- code 1bwv

/home/kaustubh/miniconda3/envs/md-analysis/lib/python3.10/site-packages/MDAnalysis/analysis/rms.py:718: DeprecationWarning: The `rmsd` attribute was deprecated in MDAnalysis 2.0.0 and will be removed in MDAnalysis 3.0.0. Please use `results.rmsd` instead.
  warnings.warn(wmsg, DeprecationWarning)


 aligned
-- code 3zxw

/home/kaustubh/miniconda3/envs/md-analysis/lib/python3.10/site-packages/MDAnalysis/analysis/rms.py:718: DeprecationWarning: The `rmsd` attribute was deprecated in MDAnalysis 2.0.0 and will be removed in MDAnalysis 3.0.0. Please use `results.rmsd` instead.
  warnings.warn(wmsg, DeprecationWarning)


 aligned
-- code 6ftl

...

/home/kaustubh/miniconda3/envs/md-analysis/lib/python3.10/site-packages/MDAnalysis/analysis/rms.py:718: DeprecationWarning: The `rmsd` attribute was deprecated in MDAnalysis 2.0.0 and will be removed in MDAnalysis 3.0.0. Please use `results.rmsd` instead.
  warnings.warn(wmsg, DeprecationWarning)


 aligned
-- code anci-wo-ssu

/home/kaustubh/miniconda3/envs/md-analysis/lib/python3.10/site-packages/MDAnalysis/analysis/rms.py:279: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(np.sum((a - b) ** 2) / N)


 aligned


/home/kaustubh/miniconda3/envs/md-analysis/lib/python3.10/site-packages/MDAnalysis/analysis/rms.py:718: DeprecationWarning: The `rmsd` attribute was deprecated in MDAnalysis 2.0.0 and will be removed in MDAnalysis 3.0.0. Please use `results.rmsd` instead.
  warnings.warn(wmsg, DeprecationWarning)


In [6]:
exp01_rmsd.to_pickle("./exp01_rmsd_rep_2.pkl")